In [ ]:
!pip install requests_html

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession

def save_to_csv(df, filename):
    df.to_csv(filename, index=False, encoding='utf-8-sig')

def get_track_id(track_url):
    return int(track_url.split('track/')[1])

def get_track_info(track_url):
    res = requests.get(track_url)
    soup = BeautifulSoup(res.text, 'html.parser')

    title = soup.select_one('#container > header > div > h1').text.strip()
    #print(soup.select_one('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(2) > td'))
    artist = soup.select_one('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(2) > td > span:nth-child(2) > a')
    if artist is not None:
        artist = artist.text.strip() # 아티스트 정보 추출
    else:
        artist = 'Unknown' # 아티스트 정보가 없을 경우 'Unknown'으로 처리
    album = soup.select_one('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.basicInfo > table > tbody > tr:nth-child(3) > td > a')
    if album is not None:
        album = album.text.strip() # 아티스트 정보 추출
    else:
        album = 'Unknown' # 아티스트 정보가 없을 경우 'Unknown'으로 처리
    like_count = soup.select_one('#container > section.sectionPadding.summaryInfo.summaryTrack > div > div.etcInfo > span > a > span > em').text.strip().replace(",", "")
    like_count = int(like_count)

    track_id = get_track_id(track_url)

    data = [[title, artist, album, track_id, like_count]]
    columns = ['title', 'artist', 'album_title', 'track_id', 'likes']
    df = pd.DataFrame(data, columns=columns)
    return df
    print(df)



#track_url = 'https://music.bugs.co.kr/track/6195812?wl_ref=list_tr_08_ab'
#df = get_track_info(track_url)
#print(df)

def get_chart_songs_info(chart_url):
    res = requests.get(chart_url)
    soup = BeautifulSoup(res.text, 'lxml')
    
    # CHARTrealtime 테이블에서 tr 태그 추출
    track_links = soup.select('#CHARTweek > table > tbody > tr')
    track_ids = [link['trackid'] for link in track_links]
    track_urls = [f"https://music.bugs.co.kr/track/{track_id}" for track_id in track_ids]

    # 추출한 tr 태그에서 trackid 정보 추출
    for link in track_links:
      track_id = link.get('trackid')
    
    # 추출한 트랙 ID를 이용한 URL 주소 생성
    track_urls = [f"https://music.bugs.co.kr/track/{link.get('trackid')}" for link in track_links]
    #print(track_urls)

    chart_df = pd.concat([get_track_info(track_url) for track_url in track_urls],
                          ignore_index=True)

    return chart_df

chart_url = 'https://music.bugs.co.kr/chart/track/week/total?chartdate=20200201' 
chart_songs_df = get_chart_songs_info(chart_url)
print(chart_songs_df)

filename = 'Bugs Music 차트30.csv'
save_to_csv(chart_songs_df, filename)



                              title             artist  \
0                            Psycho  Red Velvet (레드벨벳)   
1                            METEOR        창모(CHANGMO)   
2                          Blueming            아이유(IU)   
3                     Square (2017)   백예린 (Yerin Baek)   
4                               HIP       마마무(Mamamoo)   
..                              ...                ...   
95  미리 메리 크리스마스 (feat. 천둥 Of MBLAQ)            아이유(IU)   
96                           이상한 사람                 존박   
97                     lovelovelove   백예린 (Yerin Baek)   
98              맵고짜고단거 (Feat. 페노메코)            다이나믹 듀오   
99                        Mr.gloomy   백예린 (Yerin Baek)   

                           album_title  track_id  likes  
0           ‘The ReVe Festival’ Finale  31793255  12880  
1                              Boyhood  31769281  11516  
2                            Love poem   5803785  18186  
3             Every letter I sent you.  31780300  19976  
4            

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

import pandas as pd

# 읽어올 파일 목록
file_list = ['Bugs Music 차트.csv', 'Bugs Music 차트2.csv', 'Bugs Music 차트3.csv',
             'Bugs Music 차트4.csv', 'Bugs Music 차트5.csv', 'Bugs Music 차트6.csv',
             'Bugs Music 차트7.csv', 'Bugs Music 차트8.csv', 'Bugs Music 차트9.csv',
             'Bugs Music 차트10.csv', 'Bugs Music 차트11.csv', 'Bugs Music 차트12.csv',
             'Bugs Music 차트13.csv', 'Bugs Music 차트14.csv', 'Bugs Music 차트15.csv',
             'Bugs Music 차트16.csv', 'Bugs Music 차트17.csv', 'Bugs Music 차트18.csv',
             'Bugs Music 차트19.csv', 'Bugs Music 차트21.csv',
             'Bugs Music 차트22.csv','Bugs Music 차트23.csv']

# 파일 합치기
df_merged = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)

# 중복 제거
df_merged = df_merged.drop_duplicates()

# 결과 저장
df_merged.to_csv('Bugs Music 차트 합본.csv', index=False, encoding='utf-8-sig')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
